In [1]:
import os
import pandas as pd
import numpy as np
import subprocess
from tqdm import tqdm
import sys

from surfboard.sound import Waveform
from surfboard.feature_extraction import extract_features

In [2]:
sys.path

['/Users/mazeyu/Desktop/CMU/20fall/18797/project/code/intoxicating-voice-detection',
 '/Users/mazeyu/opt/anaconda3/envs/mlsp/lib/python38.zip',
 '/Users/mazeyu/opt/anaconda3/envs/mlsp/lib/python3.8',
 '/Users/mazeyu/opt/anaconda3/envs/mlsp/lib/python3.8/lib-dynload',
 '',
 '/Users/mazeyu/opt/anaconda3/envs/mlsp/lib/python3.8/site-packages',
 '/Users/mazeyu/opt/anaconda3/envs/mlsp/lib/python3.8/site-packages/IPython/extensions',
 '/Users/mazeyu/.ipython']

In [3]:
DATASET_PATH = "/Users/mazeyu/Desktop/CMU/20fall/18797/project/data"

DOC_PATH = 'alc_original/DOC/IS2011CHALLENGE'

DATA_PATH = 'alc_original'
TRAIN_TABLE = 'TRAIN.TBL'
D1_TABLE = 'D1.TBL'
D2_TABLE = 'D2.TBL'
TEST_TABLE = 'TESTMAPPING.txt'

In [4]:
components = ['mfcc', 
              'log_melspec',
              'magnitude_spectrum',
              'bark_spectrogram',
              'morlet_cwt',
              'chroma_stft',
              'chroma_cqt',
              'chroma_cens',
              'spectral_slope',
              'spectral_flux',
              'spectral_entropy',
              'spectral_centroid',
              'spectral_spread',
              'spectral_skewness',
              'spectral_kurtosis',
              'spectral_flatness',
              'spectral_rolloff',
              'loudness',
              'loudness_slidingwindow',
              'shannon_entropy',
              'shannon_entropy_slidingwindow',
              'zerocrossing',
              'zerocrossing_slidingwindow',
              'rms',
              'intensity',
              'crest_factor',
              'f0_contour',
              'f0_statistics',
              'ppe',
              'jitters',
              'shimmers',
              'hnr',
              'dfa',
              'lpc',
              'lsf',
              'formants',
              'formants_slidingwindow',
              'kurtosis_slidingwindow',
              'log_energy',
              'log_energy_slidingwindow']

statistics = ['max',
              'min',
              'mean',
              'std',
              'skewness',
              'kurtosis',
              'first_derivative_mean',
              'first_derivative_std',
              'first_derivative_skewness',
              'first_derivative_kurtosis',
              'second_derivative_mean',
              'second_derivative_std',
              'second_derivative_skewness',
              'second_derivative_kurtosis',
              'first_quartile',
              'second_quartile',
              'third_quartile',
              'q2_q1_range',
              'q3_q2_range',
              'q3_q1_range',
              'percentile_1',
              'percentile_99',
              'percentile_1_99_range',
              'linear_regression_offset',
              'linear_regression_slope',
              'linear_regression_mse']

In [5]:
class ALCDataset:
    def __init__(self, path):
        self.dataset_path = path
        self.__load_meta_file()

    def __process_meta(self, meta):
        meta['file_name'] = meta['file_name'].map(lambda x: x[x.find('/') + 1:].lower())
        meta['file_name'] = meta['file_name'].map(lambda x: x[:-8] + 'm' + x[-7:])
        meta['session'] = meta['file_name'].map(lambda x: x[:x.find('/')])
        meta['label'] = meta['user_state'].map(lambda x: 1 if x == 'I' else 0)
        return meta
    
    def extract_feature(self, split):
        split = split.lower()
        assert split in ('train', 'd1', 'd2', 'test')
        meta = getattr(self, f'{split}_meta')
        
        sounds = []
        for file_name in tqdm(meta['file_name']):
            sound = Waveform(path=os.path.join(self.dataset_path, DATA_PATH, file_name))
            sounds.append(sound)
        
        features_df = extract_features(sounds, components, statistics)
        features = features_df.to_numpy()
            
        np.save(f'{split}_x.npy', features)
            
        return features

    def __load_meta_file(self):
        """Load meta file.

        :return: None
        """
        assert os.path.exists(self.dataset_path)
        doc_folder = os.path.join(self.dataset_path, DOC_PATH)
        print(doc_folder)

        train_meta_path = os.path.join(doc_folder, TRAIN_TABLE)
        self.train_meta = pd.read_csv(train_meta_path, sep='\t', names=['file_name', 'bac', 'user_state'])
        self.train_meta = self.__process_meta(self.train_meta)

        d1_meta_path = os.path.join(doc_folder, D1_TABLE)
        self.d1_meta = pd.read_csv(d1_meta_path, sep='\t', names=['file_name', 'bac', 'user_state'])
        self.d1_meta = self.__process_meta(self.d1_meta)

        d2_meta_path = os.path.join(doc_folder, D2_TABLE)
        self.d2_meta = pd.read_csv(d2_meta_path, sep='\t', names=['file_name', 'bac', 'user_state'])
        self.d2_meta = self.__process_meta(self.d2_meta)

        test_meta_path = os.path.join(doc_folder, TEST_TABLE)
        self.test_meta = pd.read_csv(test_meta_path, sep='\t',
                                     names=['file_name', 'bac', 'user_state', 'test_file_name'])
        self.test_meta = self.test_meta[['file_name', 'bac', 'user_state']]
        self.test_meta = self.__process_meta(self.test_meta)

In [6]:
dataset = ALCDataset(DATASET_PATH)

/Users/mazeyu/Desktop/CMU/20fall/18797/project/data/alc_original/DOC/IS2011CHALLENGE


In [ ]:
f_train = dataset.extract_feature("train")

In [ ]:
f_d1 = dataset.extract_feature("d1")

In [ ]:
f_d2 = dataset.extract_feature("d2")

In [ ]:
f_test = dataset.extract_feature("test")